In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/historical_and_forecast_data.csv")

list=[]
for col in df.columns:
    if col not in ['index', 'Mar-25'] and not col.startswith('2'):
        list.append(col)

df_cleaned = df.drop(columns=list)

df_cleaned=df_cleaned.rename(columns={'Mar-25': 'Price_today'})

df_cleaned.head()


,index,Price_today,2025Q2,2025Q3,2025Q4,2026Q1,2026Q2,2026Q3,2026Q4,2027Q1,...,2027Q4,2028Q1,2028Q2,2028Q3,2028Q4,2029Q1,2029Q2,2029Q3,2029Q4,2030Q1
0,Albert Park-Middle Park-West St Kilda,700.0,697.421379,699.618152,694.810404,687.826767,685.415159,676.796165,679.935770,666.719495,...,664.976224,653.260323,657.406545,654.029519,663.603091,661.765671,669.900060,673.692795,682.941255,691.922189
1,Armadale,625.0,630.202314,637.680396,648.436551,652.752795,657.069039,661.385283,665.701527,670.017771,...,682.966502,687.282746,691.598990,695.915234,700.231478,704.547722,708.863966,713.180210,717.496454,721.812697
2,Carlton North,720.0,728.365576,735.993061,743.032473,749.603340,755.800894,761.701013,767.364150,772.838472,...,788.474907,793.507344,798.479158,803.402670,808.287699,813.142065,817.972001,822.782473,827.577436,832.360044
3,Carlton-Parkville,600.0,600.652519,601.556790,606.711298,601.026914,599.331589,599.695880,599.391098,599.260358,...,604.205223,608.435110,612.254466,615.899749,619.226922,623.789979,627.751477,631.401827,635.307710,638.824883
4,CBD-St Kilda Rd,650.0,642.940699,633.651440,621.726074,609.185029,596.816713,585.731950,576.569522,569.814935,...,564.710941,567.309664,571.333368,576.250730,581.535788,586.710022,591.373626,595.226278,598.076695,599.842110


In [3]:
df_avg = df_cleaned

df_avg["avg_future"]=df_avg.iloc[:,2:].mean(axis=1)


drop=[]

for col in df_avg.columns:
    if col not in ['index', 'Price_today', 'avg_future']:
        drop.append(col)

df_growth = df_avg.drop(columns=drop)

df_growth.head()

,index,Price_today,avg_future
0,Albert Park-Middle Park-West St Kilda,700.0,674.221781
1,Armadale,625.0,680.006297
2,Carlton North,720.0,784.110676
3,Carlton-Parkville,600.0,611.431314
4,CBD-St Kilda Rd,650.0,589.923889


In [4]:
df_growth['Price_change (%)'] = (df_growth['avg_future'] - df_growth['Price_today'])/df_growth['Price_today']*100

df_sorted = df_growth.sort_values(by='Price_change (%)', ascending=False)


df_sorted['index'] = df_sorted['index'].str.upper()

rental = df_sorted.rename(columns={'index': 'Suburb'})

rental.head(10) #10 suburbs with highest growth

,Suburb,Price_today,avg_future,Price_change (%)
130,MILDURA,450.0,608.481381,35.218085
108,SEAFORD-CARRUM DOWNS,550.0,726.785359,32.142793
96,BERWICK,595.0,748.595230,25.814324
137,SHEPPARTON,450.0,563.057993,25.123998
97,CRANBOURNE,580.0,715.200258,23.310389
102,PAKENHAM,530.0,652.882722,23.185419
90,CROYDON-LILYDALE,580.0,711.850817,22.732899
91,FERNTREE GULLY,580.0,706.235567,21.764753
105,FRANKSTON,530.0,638.530038,20.477366
138,SWAN HILL,400.0,480.922265,20.230566


In [5]:
rental.to_csv("../data/outputs/overall_growth.csv")

Load apartments and houses median prices 

In [76]:
df_prices = pd.read_csv("../data/raw/House_prices.csv", sep=";")

df_apartment_prices = pd.read_csv("../data/raw/apartment_prices.csv", sep=";")


Clean house prices and merge with rental 

In [81]:
merged = rental.merge(df_prices, on = 'Suburb', how='left')

columns = ['Q3_2024', 'Q4_2024']

total = merged.rename(columns={'Q1_2025':'house_price', 'Num_sales':'Num_sales_houses'})

total = total.drop(columns=columns)


Clean apartment prices

In [82]:
df_apartment_prices.head()

columns_d=['Q3_2024', 'Q4-2024']

units = df_apartment_prices.rename(columns={'Q1_2025':'unit_price'})

units = units.drop(columns=columns_d)

units.head(2)

,Suburb,unit_price,Num_sales_units
0,ABBOTSFORD,512300,86
1,ABERFELDIE,810000,2


Merge apartment prices with the rest of the data

In [83]:


total_merged = total.merge(units, on='Suburb', how='left')

data = total_merged.dropna()

data.head()

,Suburb,Price_today,avg_future,Price_change (%),house_price,Num_sales_houses,unit_price,Num_sales_units
0,MILDURA,450.0,608.481381,35.218085,485000.0,165.0,357500.0,42.0
2,BERWICK,595.0,748.595230,25.814324,900000.0,181.0,730000.0,24.0
3,SHEPPARTON,450.0,563.057993,25.123998,470500.0,188.0,460000.0,16.0
4,CRANBOURNE,580.0,715.200258,23.310389,660000.0,109.0,492100.0,24.0
5,PAKENHAM,530.0,652.882722,23.185419,670000.0,247.0,490000.0,47.0


Compute Annual Return on Investment (ROI) for houses and apartments

In [85]:
data['HOUSE Annual ROI (%)'] = data['avg_future'] / data['house_price'] * 100 
data['UNIT Annual ROI (%)'] = data['avg_future'] / data['unit_price'] * 100 

sorted_roi = data.sort_values(by = 'UNIT Annual ROI (%)', ascending=False)

sorted_roi.head(10)

/var/folders/2w/84n_l6ld0kq5y9sp9jw4q50c0000gn/T/ipykernel_57504/2318113385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HOUSE Annual ROI (%)'] = data['avg_future'] / data['house_price'] * 100
/var/folders/2w/84n_l6ld0kq5y9sp9jw4q50c0000gn/T/ipykernel_57504/2318113385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UNIT Annual ROI (%)'] = data['avg_future'] / data['unit_price'] * 100


,Suburb,Price_today,avg_future,Price_change (%),house_price,Num_sales_houses,unit_price,Num_sales_units,HOUSE Annual ROI (%),UNIT Annual ROI (%)
33,SEYMOUR,400.0,453.098796,13.274699,443000.0,27.0,222000.0,8.0,0.102280,0.204099
19,MORWELL,360.0,417.670596,16.019610,351000.0,96.0,232500.0,14.0,0.118994,0.179643
0,MILDURA,450.0,608.481381,35.218085,485000.0,165.0,357500.0,42.0,0.125460,0.170205
9,SWAN HILL,400.0,480.922265,20.230566,545000.0,35.0,299000.0,8.0,0.088243,0.160844
13,WODONGA,475.0,561.731213,18.259203,615000.0,108.0,360000.0,22.0,0.091338,0.156036
44,WEST FOOTSCRAY,540.0,601.868982,11.457219,847500.0,24.0,387800.0,34.0,0.071017,0.155201
21,DANDENONG,530.0,614.003617,15.849739,725500.0,50.0,400000.0,118.0,0.084632,0.153501
46,NORTH BENDIGO,460.0,511.675043,11.233705,497500.0,23.0,337500.0,7.0,0.102849,0.151607
22,BENALLA,440.0,508.689169,15.611175,465000.0,55.0,340000.0,9.0,0.109396,0.149614
25,LARA,540.0,620.840954,14.970547,655000.0,34.0,422500.0,8.0,0.094785,0.146945
